In [1]:
### IMPORTS ###
import numpy as np
random_seed=0
import random
random.seed(random_seed)
import scipy.io
import scipy.signal
import scipy.stats
import scipy.interpolate
import scipy.linalg
import matplotlib.pyplot as plt
from time import time
import matplotlib as mpl
mpl.rc('image', cmap='jet')
plt.rcParams.update({'font.size': 30})

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchaudio.transforms as T
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cpu")#"cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.manual_seed(0)

cpu


In [2]:
### PARAMETERS
Nz = 512 # number of discrete heights
z = np.linspace(-10, 30, Nz) # heights
epsilon = 1e-1 # thermal noise

# simulated data
Nlook = 60 # number of looks in the simulated data
Nsamples = 10000 # number of simulated samples
Nim = 6 # number of simulated images in the stack
kz = np.linspace(0.2, (Nim-1)*0.2 + 0.2, num=Nim)/2
A = np.exp(-1j*kz.reshape(-1,1)*z.reshape(-1,1).transpose())
A_simu = None
az_out_sel = 115

In [21]:
yue = scipy.io.loadmat('./biosar_profile_az116.mat', squeeze_me=True)#, simplify_cells=True) ## for newer scipy versions
print(yue.keys())
rg_out_camp = yue['rg_out_camp']-1

dict_keys(['__header__', '__version__', '__globals__', 'Rsp', 'Tomo_SP_CP', 'az_out_sel', 'kz_sel', 'psi', 'rg_out_camp', 'spec_wv', 'zw'])


In [22]:
mat = scipy.io.loadmat('./Biosar2_L_band_demo_data.mat', squeeze_me=True)#, simplify_cells=True) ## for newer scipy versions
print(mat.keys())
kz = mat['kz']
k = mat['k']
print(kz.shape)
print(k.shape)

dict_keys(['__header__', '__version__', '__globals__', 'k', 'kz'])
(935, 2553, 6)
(935, 2553, 6, 3)


In [23]:
## subset
kz_sub = kz[rg_out_camp,:200, :]
k_sub = k[rg_out_camp,:200, :, :]

print(kz_sub.shape)
print(k_sub.shape)

(826, 200, 6)
(826, 200, 6, 3)


In [24]:
mat['kz'] = kz_sub
mat['k'] = k_sub
scipy.io.savemat('./Biosar2_sample200.mat', mat)

In [25]:
mat200 = scipy.io.loadmat('./Biosar2_sample200.mat', squeeze_me=True)#, simplify_cells=True) ## for newer scipy versions
print(mat200.keys())

dict_keys(['__header__', '__version__', '__globals__', 'k', 'kz'])


In [26]:
print(np.sum(np.abs(kz_sub - mat200['kz'])))
print(np.sum(np.abs(k_sub - mat200['k'])))

0.0
0.0
